Import necessary libs

In [1]:
import os
import re
import json
import requests
import pandas as pd
import province_mapper

from datetime import datetime
from bs4 import BeautifulSoup

print('[+] Successful setup...')

[+] Successful setup...


Defining global variables

In [2]:
PROVINCES_COUNT: int = 27
DUMP_FOLDER: str = 'vhi_dump'
PROVINCES_MAP: dict = province_mapper.get_mapped_provinces()

print(json.dumps(PROVINCES_MAP, indent=4))

{
    "1": {
        "name": "Cherkasy Region",
        "id": 24
    },
    "2": {
        "name": "Chernihiv Region",
        "id": 25
    },
    "3": {
        "name": "Chernivtsi Region",
        "id": 26
    },
    "4": {
        "name": "Republic of Crimea Region",
        "id": 1
    },
    "5": {
        "name": "Dnipropetrovsk Region",
        "id": 4
    },
    "6": {
        "name": "Donetsk Region",
        "id": 5
    },
    "7": {
        "name": "Ivano-Frankivsk Region",
        "id": 9
    },
    "8": {
        "name": "Kharkiv Region",
        "id": 21
    },
    "9": {
        "name": "Kherson Region",
        "id": 22
    },
    "10": {
        "name": "Khmelnytskyi Region",
        "id": 23
    },
    "11": {
        "name": "Kyiv Region",
        "id": 10
    },
    "12": {
        "name": "Kyiv City",
        "id": 11
    },
    "13": {
        "name": "Kirovohrad Region",
        "id": 12
    },
    "14": {
        "name": "Luhansk Region",
        "id": 13
    },

Init destination folder

In [3]:
def folder_init(folder_path: str, is_silent = False) -> None:
    """
    Creates folder if it doesn't exist 
    :param folder_path: Path to the folder
    :param is_silent: If true, will hide print about existing folder!
    :return: None
    """
    
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f'[+] Created folder: {folder_path}')
    else:
        if not is_silent: 
            print(f'[x] Folder {folder_path} already exists!')
    

File downloader (download handler)

In [4]:
def fetch(province_id: int, start_year: int, end_year: int) -> None:
    """
    Fetches mean data from NOAA 
    :param province_id: Province ID based on the NOAA mapping
    :param start_year: The starting year for the requested time series data Format: (YYYY)
    :param end_year: The ending year for the requested time series data. Format: (YYYY)
    :return: None
    """
    
    if not validate_id(province_id):
        print('[!] Invalid province ID')
        return
    
    mapped_province_id = PROVINCES_MAP[province_id]["id"]
    mapped_province_name = PROVINCES_MAP[province_id]["name"]
    
    start_year, end_year = validate_years(start_year, end_year)
    
    if start_year is None or end_year is None: 
        return

    print(f'[*] Fetching data:\n\t'
          f'Province ID: {province_id} | {mapped_province_name}\n\t'
          f'Start year: {start_year}\n\t'
          f'End year: {end_year}')

    url: str = f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1={start_year}&year2={end_year}&type=Mean'
    
    try:
        req = requests.get(url)
        
        if req.status_code != 200:
            print('[!] Request failed!')
            return
        
        # clearing response from html tags
        soup = BeautifulSoup(req.text, "html.parser")
        clear_data = soup.get_text()
        
        # assembling the file name
        timestamp = datetime.now().strftime('%Y-%m-%dT%H-%M')
        filename = f'vhi_{mapped_province_id}_{timestamp}.csv'
        
        # Deleting last comma in the data series that makes useless shift
        clear_data = re.sub(r',\s*$', '', clear_data, flags=re.MULTILINE)
        
        # changing province id based on the real mapping
        clear_data = re.sub(r"Province= \d+:", f"Province= {mapped_province_id}:", clear_data)
        print(f'[!] Province ID was changed to the real mapping: {province_id} (NOAA) -> {mapped_province_id} (UKR oblast centers mapping)')
        
        save(directory=DUMP_FOLDER, filename=filename, data=clear_data)
        #print(clear_data)
    
    except Exception as err:
        raise Exception(f'[!] Error happened: {err}')


Function for years validation 

In [5]:
def validate_years(start_year: int, end_year: int) -> (int, int):
    """
    Checks if start_year and end_year are within the correct range
    :param start_year: The starting year for the requested time series data Format: (YYYY)
    :param end_year: The ending year for the requested time series data. Format: (YYYY). Set end_year to 9999 to validate only start_year for the format.
    :return: A tuple of two integers if start_year and end_year passed all checks. 
    """
    current_year:int = datetime.today().year
    year_pattern = r'^\d{4}$'
    
    if not re.match(year_pattern, str(start_year)) or not re.match(year_pattern, str(end_year)):
        print(f'[!] Invalid year format. Only allowed one is: YYYY (4 integers)')
        return None, None
    
    if end_year == 9999 and start_year:
        print(start_year)
        return start_year
    
    if start_year < 2013:
        print(f'[!] Start year should be greater or equal to 2013!')
        return None, None
    
    if end_year >= current_year:
        print(f'[!] For full data coverage, end_year({end_year}) should be less than current_year({current_year})!')
        return None, None
    
    if start_year > end_year:
        print(f'[!] start_year({start_year}) cannot be greater than end_year({end_year})')
        return None, None
    
    print(f'[+] Successful years range validation!')
    return start_year, end_year
    

Function for ID validation

In [6]:
def validate_id(province_id: int) -> bool:
    """
    Checks if given province_id is in the mapped dictionary
    :param province_id: Province ID based on the NOAA mapping
    :return: Bool 
    """
    
    if province_id not in PROVINCES_MAP.keys():
        return False
    
    print(f'[+] Successful ID validation!')
    return True
    

File saver

In [7]:
def save(directory: str, filename: str, data: str) -> None:
    """
    Saves given data to the file in the specific dir
    :param directory: Directory where the data is saved.
    :param filename: Name of the file to save the data to.
    :param data: Data to save.
    :return: 
    """
    
    folder_init(directory, is_silent=True)
    
    with open(f'{directory}/{filename}', 'w') as file:
        file.write(data)
        print(f'[+] Saved {filename} to {directory}/')
        
        file.close()
    

Bulk file downloader

In [8]:
def fetch_bulk(start_id: int, end_id: int, start_year: int, end_year: int) -> None:
    """
    Using fetch() to bulk-download VHI data for the specific range of regions
    :param start_id: The starting province(region) ID (NOAA list) to download
    :param end_id: The ending province(region) ID (NOAA list) to download
    :param start_year: The starting year for the requested time series data Format: (YYYY)
    :param end_year: The ending year for the requested time series data. Format: (YYYY)
    :return: None
    """
    
    for i in range(start_id, end_id + 1):
        print(f'[*] Fetch order: {i}')
        try: 
            fetch(i, start_year, end_year)
        except Exception as err:
            print(f'[!] Error fetching: {err}')


Essentially, clears given directory (dump_folder, most of the time)

In [9]:
def clear_dump_folder(directory: str) -> None:
    """
    Cleans dump folder
    :param directory: Path to the folder
    :return: None
    """
    
    if not os.path.exists(directory):
        print(f'[!] Directory does not exist!')
        return
    
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
                print(f'[*] Deleted file: {file_path}')
        except Exception as err:
             print(f"[!] Failed to delete {file_path}. Reason: {err}")


Function that creates dataframe from the `.csv` files in the given directory

In [10]:
def get_dataframe(directory: str) -> pd.DataFrame | None:
    """
    Creates dataframe from the files in the directory
    :param directory: Path to the directory
    :return: Pandas dataframe
    """
    
    headers: list = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI']
    dataframes: list = []
    
    if not os.path.exists(directory):
        print(f'[!] Directory does not exist!')
        return
    
    for file in os.listdir(directory):
        if not file.endswith('.csv'):
            print(f'[!] File ({file}) is not a .csv. Skipping it!')
            continue
        file_path = os.path.join(directory, file)
        
        try:
            df = pd.read_csv(file_path, header=1, names=headers, skiprows=1)
            province_id = int(file.split('_')[1])
            df.insert(0, 'PID', province_id, True)
            dataframes.append(df)
        except Exception as err:
            print(f'[!] Failed to read {err}')
    
    if not dataframes:
        raise Exception('No files found!')
        
    return pd.concat(dataframes, ignore_index=True) 
    

Main entry point for the script. I love main(), even in the py notebook.... yes

In [11]:
# clearing dump folder
clear_dump_folder(DUMP_FOLDER)
    
# bulk fetch for provinces from 1 to 4 (NOAA id list)
fetch_bulk(1, PROVINCES_COUNT, 2013, 2023)
    
# collecting files into the pandas dataframe
df = get_dataframe(DUMP_FOLDER)

[*] Deleted file: vhi_dump/vhi_25_2024-04-16T21-48.csv
[*] Deleted file: vhi_dump/vhi_24_2024-04-16T21-48.csv
[*] Deleted file: vhi_dump/vhi_21_2024-04-16T21-48.csv
[*] Deleted file: vhi_dump/vhi_9_2024-04-16T21-48.csv
[*] Deleted file: vhi_dump/vhi_5_2024-04-16T21-48.csv
[*] Deleted file: vhi_dump/vhi_22_2024-04-16T21-48.csv
[*] Deleted file: vhi_dump/vhi_23_2024-04-16T21-48.csv
[*] Deleted file: vhi_dump/vhi_4_2024-04-16T21-48.csv
[*] Deleted file: vhi_dump/vhi_26_2024-04-16T21-48.csv
[*] Deleted file: vhi_dump/vhi_1_2024-04-16T21-48.csv
[*] Fetch order: 1
[+] Successful ID validation!
[+] Successful years range validation!
[*] Fetching data:
	Province ID: 1 | Cherkasy Region
	Start year: 2013
	End year: 2023
[!] Province ID was changed to the real mapping: 1 (NOAA) -> 24 (UKR oblast centers mapping)
[+] Saved vhi_24_2024-04-17T12-41.csv to vhi_dump/
[*] Fetch order: 2
[+] Successful ID validation!
[+] Successful years range validation!
[*] Fetching data:
	Province ID: 2 | Chernihiv 

In [12]:
df

,PID,Year,Week,SMN,SMT,VCI,TCI,VHI
0,21,2013,2,0.042,253.42,53.83,59.87,56.85
1,21,2013,3,0.042,253.26,52.90,60.89,56.90
2,21,2013,4,0.045,253.89,55.97,63.09,59.53
3,21,2013,5,0.050,256.28,55.71,59.10,57.40
4,21,2013,6,0.058,259.88,55.67,48.78,52.22
...,...,...,...,...,...,...,...,...
14841,22,2023,48,0.101,277.36,37.50,15.86,26.68
14842,22,2023,49,0.097,275.73,38.12,16.77,27.45
14843,22,2023,50,0.097,275.42,40.98,13.16,27.07
14844,22,2023,51,0.097,274.98,43.98,11.14,27.56


Function that returns VHI series for sepecific region within specific year 

In [13]:
def get_vhi(dataframe: pd.DataFrame, province_id: int, year: int) -> pd.DataFrame | None:
    """
    Returns VHI series for the specified region and year.
    
    :param dataframe: Pandas DataFrame containing the data.
    :param province_id: Region ID as per 'PID' column in the DataFrame.
    :param year: Year as integer to filter the VHI series.
    :return: Pandas Series containing VHI values for the specified PID and year.
    """
    
    if province_id not in dataframe['PID'].unique():
        print(f"[!] PID {province_id} not found in the DataFrame.")
        return None

    if year not in df['Year'].unique():
        print(f"[!] Year {year} not found in the DataFrame.")
        return None
    
    print(f' VHI data series for {province_id=} and {year=}')
    
    temp = dataframe[(dataframe['PID'] == province_id) & (dataframe['Year'] == year)]
    
    return temp['VHI'].reset_index(drop=True)
    

In [14]:
get_vhi(df, 1, 2022)

 VHI data series for province_id=1 and year=2022


0     44.28
1     45.61
2     44.44
3     40.63
4     40.24
5     40.19
6     41.37
7     41.80
8     41.67
9     42.30
10    43.03
11    43.30
12    43.96
13    44.50
14    45.52
15    48.08
16    53.09
17    57.25
18    60.03
19    60.43
20    59.39
21    60.43
22    60.93
23    60.22
24    60.07
25    59.63
26    59.18
27    58.56
28    58.15
29    52.95
30    51.57
31    49.29
32    46.48
33    43.41
34    40.62
35    37.58
36    36.78
37    37.88
38    39.34
39    40.95
40    41.73
41    42.07
42    40.80
43    37.01
44    34.83
45    35.12
46    35.85
47    38.75
48    40.24
49    41.19
50    42.21
51    41.77
Name: VHI, dtype: float64

In [15]:
def get_max_min_vhi(dataframe: pd.DataFrame, provinces: list, years: list) -> pd.DataFrame | None:
    """
    Returns Min VHI and Max VHI series for the specified provinces and years
    :param dataframe: Pandas DataFrame containing the data.
    :param provinces: List of provinces id (real regional mapping)
    :param years: List of years to retrieve
    :return: Pandas DataFrame or Nothing if error occurs
    """
    
    if not set(provinces).issubset(dataframe['PID'].unique()):
        print("[!] One or more specified provinces are not in the DataFrame.")
        print(f'[?] Possible PID in the given DateFrame: {suggest_values(dataframe)[0]}')
        return None

    if not set(years).issubset(dataframe['Year'].unique()):
        print("[!] One or more specified years are not in the DataFrame.")
        print(f'[?] Possible years in the given DateFrame: {suggest_values(dataframe)[1]}')
        return None
    
    result:list = []
    
    for pid in provinces: 
        for year in sorted(years):
            
            temp = dataframe[(dataframe['PID'] == pid) & (dataframe['Year'] == year)]
            
            if temp.empty:
                continue
            
            min_vhi = temp['VHI'].min()
            max_vhi = temp['VHI'].max()
            
            result.append({'Province ID': pid, 
                                    'Year': year, 
                                    'Min VHI': min_vhi, 
                                    'Max VHI': max_vhi})
    
    result = pd.DataFrame(result)
    
    return result if not result.empty else None
    
    

In [16]:
get_max_min_vhi(df, [1, 22, 24], [2019, 2022, 2014, 2017])

,Province ID,Year,Min VHI,Max VHI
0,1,2014,28.95,56.51
1,1,2017,21.17,60.38
2,1,2019,18.85,58.77
3,1,2022,34.83,60.93
4,22,2014,26.24,53.41
5,22,2017,24.72,58.49
6,22,2019,28.06,61.08
7,22,2022,35.70,54.81
8,24,2014,37.59,67.44
9,24,2017,35.58,59.50


Some util function that suggests years and province ids based on the provided dataframe 

In [17]:
def suggest_values(dataframe: pd.DataFrame) -> (list, list):
    """
    Returns the unique years and province IDs from the DataFrame
    :param dataframe: Pandas DataFrame
    :return: A tuple containing two lists of years and province ids
    """
    
    if dataframe.empty:
        print(f'[!] DataFrame is empty :/')
        return 
    
    years = dataframe['Year'].unique().tolist()
    ids = dataframe['PID'].unique().tolist()
    
    years.sort()
    ids.sort()
    
    return ids, years

Returns VHI series for the specified provinces and years

In [18]:
def get_vhi_range(dataframe: pd.DataFrame, provinces: list, years: list) -> pd.DataFrame | None:
    """
    Returns VHI series for the specified provinces and years
    :param dataframe: Pandas DataFrame containing the data.
    :param provinces: List of provinces id (real regional mapping)
    :param years: List of years to retrieve
    :return: Pandas DataFrame or Nothing if error occurs
    """
    
    if not set(provinces).issubset(dataframe['PID'].unique()):
        print("[!] One or more specified provinces are not in the DataFrame.")
        print(f'[?] Possible PID in the given DateFrame: {suggest_values(dataframe)[0]}')
        return None

    if not set(years).issubset(dataframe['Year'].unique()):
        print("[!] One or more specified years are not in the DataFrame.")
        print(f'[?] Possible years in the given DateFrame: {suggest_values(dataframe)[1]}')
        return None
    
    result: pd.DataFrame = dataframe[dataframe['PID'].isin(provinces) & dataframe['Year'].isin(years)]
    
    if result.empty:
        print("[!] Some error happened, so no data is returned.")
        return
    
    result = result[['PID', 'Year', 'Week', 'VHI']]
    
    return result
    

In [19]:
get_vhi_range(df, [1, 22, 24], [2022, 2019])

,PID,Year,Week,VHI
3166,24,2019,1,52.26
3167,24,2019,2,53.94
3168,24,2019,3,56.98
3169,24,2019,4,56.87
3170,24,2019,5,52.78
...,...,...,...,...
14789,22,2022,48,41.71
14790,22,2022,49,42.46
14791,22,2022,50,42.47
14792,22,2022,51,43.47


In [20]:
def analyse(dataframe: pd.DataFrame, vhi_threshold: int, affected: int) -> pd.DataFrame | None:
    """
    Analyze years when extreme droughts affected more than a specified percentage of provinces.
    
    :param dataframe: 
    :param vhi_threshold: 
    :param affected: 
    :return: 
    """
    
    temp = dataframe[dataframe['VHI'] <= vhi_threshold]
    
    if temp.empty:
        print(f"[!] There are no entries in the DataFrame with VHI below or equal to the given threshold({vhi_threshold}) value")
        return
    
    temp_year = temp.groupby('Year')['PID'].nunique()
    
    total = dataframe['PID'].nunique()
    
    min_affected_provinces = int(total * affected / 100)
    
    notable = temp_year[temp_year >= min_affected_provinces].index.tolist()
    
    result = temp[temp['Year'].isin(notable)]

    return result[['PID', 'Year', 'Week', 'VHI']]
    

In [22]:
analyse(df, 20, 20)

,PID,Year,Week,VHI
5538,8,2020,37,18.27
5539,8,2020,38,18.43
9534,1,2020,36,18.93
9535,1,2020,37,18.79
9536,1,2020,38,19.82
11248,4,2020,37,19.78
11249,4,2020,38,19.40
12390,15,2020,37,19.34
12391,15,2020,38,19.28
13532,5,2020,37,18.58
